In [1]:
from src.arguments import ModelArguments, DataArguments
from src.model.model import MMEBModel
from src.model.processor import load_processor, QWEN2_VL, VLM_IMAGE_TOKENS, \
    Qwen2_VL_process_fn, LLAVA_QWEN2, FastVLM_process_fn
from src.utils import batch_to_device
from PIL import Image
import numpy as np
import torch
import math
%matplotlib inline
import matplotlib.pyplot as plt
import torch.nn.functional as F

from transformers.image_transforms import (
    convert_to_rgb,
    resize,
)

/workspace/VLM_Embed/vlm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/VLM_Embed/vlm/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/workspace/VLM_Embed/src/model/vlm_backbone/internvideo2/modeling_internvideo2.py:541: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


FusedMLP of flash_attn is not installed!!!
DropoutAddRMSNorm of flash_attn is not installed!!!
flash_attn_interface or bert_padding of flash_attn is not installed!!!


[2026-01-02 20:00:52,731] DEBUG [matplotlib:342] matplotlib data path: /workspace/VLM_Embed/vlm/lib/python3.12/site-packages/matplotlib/mpl-data
[2026-01-02 20:00:52,737] DEBUG [matplotlib:342] CONFIGDIR=/root/.config/matplotlib
[2026-01-02 20:00:52,751] DEBUG [matplotlib:1560] interactive is False
[2026-01-02 20:00:52,752] DEBUG [matplotlib:1561] platform is linux
[2026-01-02 20:00:52,774] DEBUG [matplotlib:342] CACHEDIR=/root/.cache/matplotlib
[2026-01-02 20:00:52,776] DEBUG [matplotlib.font_manager:1635] Using fontManager instance from /root/.cache/matplotlib/fontlist-v390.json
[2026-01-02 20:00:52,967] DEBUG [matplotlib.pyplot:496] Loaded backend module://matplotlib_inline.backend_inline version unknown.
[2026-01-02 20:00:52,970] DEBUG [matplotlib.pyplot:496] Loaded backend inline version unknown.


In [2]:
# model_args = ModelArguments(
#     model_name='Qwen/Qwen2-VL-2B',
#     checkpoint_path='TIGER-Lab/VLM2Vec-Qwen2VL-2B',
#     pooling='last',
#     normalize=True,
#     model_backbone='qwen2_vl',
#     lora=True
# )
model_args = ModelArguments(
    model_name='apple/FastVLM-0.5B',
    pooling='last',
    normalize=True,
    model_backbone=LLAVA_QWEN2,
)
data_args = DataArguments()

processor = load_processor(model_args, None)
model = MMEBModel.build(model_args)
model = model.to('cuda', dtype=torch.bfloat16)
model.eval()

[2026-01-02 20:00:52,982] INFO [src.utils:21] Loading processor from: apple/FastVLM-0.5B
[2026-01-02 20:00:52,986] DEBUG [urllib3.connectionpool:1049] Starting new HTTPS connection (1): huggingface.co:443


Processor load here for LLAVA-QWEN2


[2026-01-02 20:00:53,273] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/tokenizer_config.json HTTP/1.1" 307 0
[2026-01-02 20:00:53,285] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/apple/FastVLM-0.5B/16375720c2d673fa583e57e9876afde27549c7d0/tokenizer_config.json HTTP/1.1" 200 0
[2026-01-02 20:00:53,520] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "GET /api/models/apple/FastVLM-0.5B/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64
[2026-01-02 20:00:54,220] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/config.json HTTP/1.1" 307 0
[2026-01-02 20:00:54,231] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/apple/FastVLM-0.5B/16375720c2d673fa583e57e9876afde27549c7d0/config.json HTTP/1.1" 200 0
[2026-01-02 20:00:54,464] DEBUG [urllib3.connect

Detected model type: llava_qwen2
Determined model backbone: llava_qwen2


[2026-01-02 20:00:55,670] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/generation_config.json HTTP/1.1" 307 0
[2026-01-02 20:00:55,682] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/apple/FastVLM-0.5B/16375720c2d673fa583e57e9876afde27549c7d0/generation_config.json HTTP/1.1" 200 0
[2026-01-02 20:00:55,903] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0


MMEBModel(
  (encoder): LlavaQwen2ForCausalLM(
    (model): LlavaQwen2Model(
      (embed_tokens): Embedding(151936, 896)
      (layers): ModuleList(
        (0-23): 24 x Qwen2DecoderLayer(
          (self_attn): Qwen2Attention(
            (q_proj): Linear(in_features=896, out_features=896, bias=True)
            (k_proj): Linear(in_features=896, out_features=128, bias=True)
            (v_proj): Linear(in_features=896, out_features=128, bias=True)
            (o_proj): Linear(in_features=896, out_features=896, bias=False)
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
            (up_proj): Linear(in_features=896, out_features=4864, bias=False)
            (down_proj): Linear(in_features=4864, out_features=896, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
          (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        )
      )
   

In [3]:
processor_inputs = {
    "text": [f'{VLM_IMAGE_TOKENS[LLAVA_QWEN2]} Represent the given image with the following question: What is in the image',
          f'{VLM_IMAGE_TOKENS[LLAVA_QWEN2]} Represent the given image with the following question: What is in the image'],
    "images": [Image.open('example.jpg').resize((500, 1025)),
            Image.open('example.jpg')],
}

inputs = FastVLM_process_fn(
    processor_inputs,
    processor, 
    # square_padding=True
    )
inputs = batch_to_device(inputs, "cuda")
inputs['images'][0].shape, inputs['images'][1].shape



(torch.Size([3, 1024, 512]), torch.Size([3, 768, 1024]))

In [6]:
type(model.encoder.get_vision_tower().vision_tower.model)

src.model.llava.model.multimodal_encoder.mobileclip.mci.FastViT

In [ ]:
output = model.encode_input(inputs)
output

(tensor([[-0.0052, -0.0527,  0.0134,  ..., -0.0244, -0.0181, -0.0091],
         [ 0.0242, -0.0060,  0.0093,  ..., -0.0164, -0.0190,  0.0400]],
        device='cuda:0', dtype=torch.bfloat16, grad_fn=<DivBackward0>),
 [tensor([[ 0.3379, -1.3906,  0.1826,  ..., -0.4785,  0.4219, -0.0588],
          [ 0.0327, -0.0070,  0.0464,  ...,  0.3047, -0.2021,  0.0119],
          [ 0.1084, -0.1523, -0.1875,  ..., -0.0908, -0.2041, -0.3496],
          ...,
          [-0.0027, -0.0918, -0.5781,  ...,  0.4883, -0.2119, -0.4531],
          [ 0.0723, -0.0354, -0.3379,  ...,  0.3398, -0.1924, -0.1963],
          [ 0.1484, -0.2402, -0.0610,  ...,  0.0295,  0.1216,  0.0659]],
         device='cuda:0', dtype=torch.bfloat16, grad_fn=<ViewBackward0>),
  tensor([[ 0.4453, -1.5234,  0.1592,  ..., -0.5625,  0.4551, -0.0903],
          [ 0.0601,  0.0139,  0.0923,  ...,  0.4102, -0.2148, -0.0045],
          [ 0.0913, -0.1748, -0.1079,  ..., -0.1328, -0.1904, -0.3535],
          ...,
          [-0.0205, -0.1406, -0.

In [ ]:
x = torch.randn(1, 3, 768, 768).to('cuda', dtype=torch.bfloat16)
with torch.no_grad():
    y = model.encoder.get_vision_tower()(x)
y.shape

RuntimeError: Given input size: (3072x12x12). Calculated output size: (3072x0x0). Output size is too small